In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import pandas_profiling 

# 전처리
from sklearn.preprocessing import LabelEncoder, StandardScaler

# 데이터 분할
from sklearn.model_selection import StratifiedKFold, KFold, GridSearchCV

# 모델링
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

# 평가지표
from sklearn.metrics import accuracy_score, mean_squared_error

In [35]:
loan_data=pd.read_csv(r'C:\Users\KYOBO\Desktop\dataset\loan_result.csv')
log_data=pd.read_csv(r'C:\Users\KYOBO\Desktop\dataset\log_data.csv')
spec_data=pd.read_csv(r'C:\Users\KYOBO\Desktop\dataset\user_spec.csv')

### loan_data test,train split

In [36]:
loan_target=loan_data[loan_data.is_applied.isnull()]

In [37]:
loan_target

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied
0,1748340,2022-06-07 13:05,7,191,42000000.0,13.6,NaN
1,1748340,2022-06-07 13:05,25,169,24000000.0,17.9,NaN
2,1748340,2022-06-07 13:05,2,7,24000000.0,18.5,NaN
3,1748340,2022-06-07 13:05,4,268,29000000.0,10.8,NaN
4,1748340,2022-06-07 13:05,11,118,5000000.0,16.4,NaN
...,...,...,...,...,...,...,...
1048570,1398164,2022-06-22 11:15,23,108,23000000.0,13.7,NaN
1048571,1398164,2022-06-22 11:15,1,61,4000000.0,13.9,NaN
1048572,1398164,2022-06-22 11:15,7,220,7000000.0,12.4,NaN
1048573,1398164,2022-06-22 11:15,62,200,25000000.0,11.8,NaN


In [38]:
loan_train=loan_data[(loan_data['is_applied']==0)|(loan_data['is_applied']==1)]

In [39]:
loan_train

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied
13284,2157865,2022-05-09 8:44,54,235,20000000.0,16.5,1.0
13285,576643,2022-05-09 10:54,54,235,11000000.0,16.5,0.0
13286,576643,2022-05-09 10:54,11,118,3000000.0,20.0,0.0
13287,2136706,2022-05-09 10:41,42,216,10000000.0,13.5,0.0
13288,2136706,2022-05-09 10:41,25,169,22000000.0,15.9,0.0
...,...,...,...,...,...,...,...
1048538,422549,2022-03-30 23:29,55,12,18000000.0,13.5,1.0
1048539,422549,2022-03-30 23:29,35,107,5000000.0,14.7,0.0
1048540,422549,2022-03-30 23:29,13,262,20000000.0,16.1,0.0
1048541,422549,2022-03-30 23:29,15,204,2000000.0,15.5,0.0


In [40]:
log_data

,user_id,event,timestamp,mp_os,mp_app_version,date_cd
0,576409,StartLoanApply,2022-03-25 11:12,Android,3.8.2,2022-03-25
1,576409,ViewLoanApplyIntro,2022-03-25 11:12,Android,3.8.2,2022-03-25
2,72878,EndLoanApply,2022-03-25 11:14,Android,3.8.4,2022-03-25
3,645317,OpenApp,2022-03-25 11:15,iOS,3.6.1,2022-03-25
4,645317,UseLoanManage,2022-03-25 11:15,iOS,3.6.1,2022-03-25
...,...,...,...,...,...,...
1048570,636267,OpenApp,2022-03-16 18:11,Android,3.8.2,2022-03-16
1048571,76065,OpenApp,2022-03-16 18:11,Android,3.8.1,2022-03-16
1048572,636267,Login,2022-03-16 18:11,Android,3.8.2,2022-03-16
1048573,636267,UseLoanManage,2022-03-16 18:11,Android,3.8.2,2022-03-16


In [70]:
log_data=log_data.sort_values(by=['timestamp'])

In [71]:
log_data

,user_id,event,timestamp,mp_os,mp_app_version,date_cd
2809,512341,7,2022-03-01 0:00,Android,3.8.0,2022-03-01
281253,122253,5,2022-03-01 0:00,Android,3.7.0,2022-03-01
473942,140973,5,2022-03-01 0:00,Android,3.8.0,2022-03-01
473941,140973,6,2022-03-01 0:00,Android,3.8.0,2022-03-01
473944,140973,5,2022-03-01 0:00,Android,3.8.0,2022-03-01
...,...,...,...,...,...,...
189165,292903,1,2022-06-30 9:59,Android,3.15.1,2022-06-30
189167,292903,2,2022-06-30 9:59,Android,3.15.1,2022-06-30
189163,72155,6,2022-06-30 9:59,Android,3.15.1,2022-06-30
185349,402172,8,2022-06-30 9:59,Android,3.11.1,2022-06-30


In [83]:
log_data=log_data.sort_values(by=["user_id","timestamp","event"], ascending=[True,True,True])

In [84]:
log_data

,user_id,event,timestamp,mp_os,mp_app_version,date_cd
15177,12,1,2022-04-16 17:01,Android,3.9.1,2022-04-16
15178,12,2,2022-04-16 17:01,Android,3.9.1,2022-04-16
15181,12,4,2022-04-16 17:01,Android,3.9.1,2022-04-16
15180,12,5,2022-04-16 17:01,Android,3.9.1,2022-04-16
15179,12,6,2022-04-16 17:01,Android,3.9.1,2022-04-16
...,...,...,...,...,...,...
603420,879693,6,2022-05-20 16:36,iOS,3.10.2,2022-05-20
603421,879693,7,2022-05-20 16:36,iOS,3.10.2,2022-05-20
603422,879693,8,2022-05-20 16:36,iOS,3.10.2,2022-05-20
603428,879693,6,2022-05-20 16:39,iOS,3.10.2,2022-05-20


In [43]:
#user_id = log_data['user_id'][0]
#for i in range(len(log_data)):
    #user_id_=log_data['user_id'][i]
    #if user_id==user_id_:
        #user_id=user_id_
    #else:
        #break#

In [44]:
#user_id

In [45]:
log_data['event'].value_counts()

OpenApp                    230390
EndLoanApply               178491
Login                      164551
StartLoanApply             125818
ViewLoanApplyIntro         120588
UseLoanManage              107556
CompleteIDCertification     82151
GetCreditInfo               35759
SignUp                       2134
UsePrepayCalc                 690
UseDSRCalc                    447
Name: event, dtype: int64

In [50]:
log_data['event'].replace({'OpenApp':1,'Login':2,'SignUp':3}, inplace=True)

In [51]:
log_data['event'].value_counts()

1                          230390
EndLoanApply               178491
2                          164551
StartLoanApply             125818
ViewLoanApplyIntro         120588
UseLoanManage              107556
CompleteIDCertification     82151
GetCreditInfo               35759
3                            2134
UsePrepayCalc                 690
UseDSRCalc                    447
Name: event, dtype: int64

In [56]:
log_data['event'].replace({'CompleteIDCertification':4,'ViewLoanApplyIntro':5,'StartLoanApply':6}, inplace=True)

In [57]:
log_data['event'].value_counts()

1                230390
7                178491
2                164551
6                125818
5                120588
8                107556
4                 82151
GetCreditInfo     35759
3                  2134
9                   690
UseDSRCalc          447
Name: event, dtype: int64

In [58]:
log_data['event'].replace({'EndLoanApply':7,'UseLoanManage':8,'UsePrepayCalc':9}, inplace=True)

In [59]:
log_data['event'].value_counts()

1                230390
7                178491
2                164551
6                125818
5                120588
8                107556
4                 82151
GetCreditInfo     35759
3                  2134
9                   690
UseDSRCalc          447
Name: event, dtype: int64

In [61]:
log_data['event'].replace({'UseDSRCalc':10,'GetCreditInfo':11}, inplace=True)

In [62]:
log_data['event'].value_counts()

1     230390
7     178491
2     164551
6     125818
5     120588
8     107556
4      82151
11     35759
3       2134
9        690
10       447
Name: event, dtype: int64

In [64]:
log_data['event'].dtype

dtype('O')

In [65]:
log_data

,user_id,event,timestamp,mp_os,mp_app_version,date_cd
15179,12,6,2022-04-16 17:01,Android,3.9.1,2022-04-16
15180,12,5,2022-04-16 17:01,Android,3.9.1,2022-04-16
15178,12,2,2022-04-16 17:01,Android,3.9.1,2022-04-16
15181,12,4,2022-04-16 17:01,Android,3.9.1,2022-04-16
15182,12,7,2022-04-16 17:03,Android,3.9.1,2022-04-16
...,...,...,...,...,...,...
603415,879693,5,2022-05-20 16:31,iOS,3.10.2,2022-05-20
603418,879693,4,2022-05-20 16:31,iOS,3.10.2,2022-05-20
603419,879693,7,2022-05-20 16:33,iOS,3.10.2,2022-05-20
603421,879693,7,2022-05-20 16:36,iOS,3.10.2,2022-05-20
